In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data/esa_csvs_raw.txt', sep='\t')

In [4]:
df

,csvFileName,csvText
0,1059614_14_lattice-v_1.csv,"[[""Survey Date"", ""Aboriginal Communities"", ""De..."
1,1059614_17_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
2,1059614_18_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
3,1059614_19_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
4,1059614_20_lattice-v_1.csv,"[[""Field Site No."", ""Name, Legal Location"", ..."
...,...,...
52286,895339_38_lattice-v_1.csv,"[[""KP"", ""Legal Land Location (W4M)"", ""Land Use..."
52287,895339_45_lattice-v_1.csv,"[[""Common Name Scientific Name Signs Used to I..."
52288,895339_46_lattice-v_1.csv,"[[""Common Name"", ""Scientific Name"", ""Signs Use..."
52289,895339_47_lattice-v_1.csv,"[[""Common Name"", ""Scientific Name"", ""Signs Use..."


In [5]:
df['csvText'] = df['csvText'].replace('[^a-zA-Z0-9 ]', ' ', regex=True)
df['csvText'] = df['csvText'].replace(' +', ' ', regex=True)

In [6]:
df.tail()

,csvFileName,csvText
52286,895339_38_lattice-v_1.csv,KP Legal Land Location W4M Land Use s Comment...
52287,895339_45_lattice-v_1.csv,Common Name Scientific Name Signs Used to Ide...
52288,895339_46_lattice-v_1.csv,Common Name Scientific Name Signs Used to Ide...
52289,895339_47_lattice-v_1.csv,Common Name Scientific Name Signs Used to Ide...
52290,895339_7_lattice-v_1.csv,56 56 Morinville552627252423222155LamontFort ...


In [7]:
df_index = pd.read_csv('./data/ESA_website_ENG_2021_03_04_final.csv')

In [8]:
df_index.rename(columns={"Old CSV Filename": "csvFileName"}, inplace=True)

In [9]:
df_index

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28886,38020,TABLE 1 GENERAL LAND USE AND ENVIRONMENTAL SET...,Table,Application for the Edmonton to Hardisty Pipel...,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst/dm...,10,47.0,2.41,Yes,dmntnhrdst,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst.zip,False,dmntnhrdst_table-1-general-land-use-and-enviro...,895339_10_lattice-v_1.csv
28887,38021,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Table,Application for the Edmonton to Hardisty Pipel...,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst/dm...,12,47.0,2.41,Yes,dmntnhrdst,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst.zip,False,dmntnhrdst_table-2-occurrences-of-species-with...,895339_12_lattice-v_1.csv
28888,38022,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Table,Application for the Edmonton to Hardisty Pipel...,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst/dm...,13,47.0,2.41,Yes,dmntnhrdst,http://www.cer-rec

In [59]:
df_merged = df[df['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [60]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [61]:
df_joined

,Index,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,csvText
0,9134,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,Survey Date Aboriginal Communities De...
1,9135,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,Field Site No Name Legal Location ...
2,9136,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,Field Site No Name Legal Location ...
3,9137,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,Field Site No Name Legal Location ...
4,9138,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,Field Site No Name Legal Location ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28886,38020,TABLE 1 GENERAL LAND USE AND ENVIRONMENTAL SET...,Table,Application for the Edmonton to Hardisty Pipel...,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,10,47.0,2.41,Yes,dmntnhrdst,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst.zip,False,dmntnhrdst_table-1-general-land-use-and-enviro...,895339_10_lattice-v_1.csv,Setting Information Project Details ...
28887,38021,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Table,Application for the Edmonton to Hardisty Pipel...,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,12,47.0,2.41,Yes,dmntnhrdst,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst.zip,False,dmntnhrdst_table-2-occurrences-of-species-with...,895339_12_lattice-v_1.csv,Species Common Name Scientific Name F...
28888,38022,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Table,Application for the Edmonton to Hardisty Pipel...,Edmonton to Hardisty,12/14/2012,Enbridge Pipelines Inc.,Oil,B2-32 - Volume IIA - ESA - Appendix 9 - Wildli...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,13,47.0,2.41,Yes,dmntnhrdst,http://www.cer-rec.gc.ca/esa-ees/dmntnhrdst.zip,False,dmntnhrdst_table-2-occurrences-of-species-with...,895339_13_lattice-v_1.csv,Species Common Name Sci

In [62]:
del df
del df_index
del df_merged

In [63]:
df_joined.to_csv('./data/esa_index_with_table_text_no_labels.csv')